# Estrategias

- Un solo link de conexión

In [6]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Encuentra el elemento más cercano en un solo bucle
def find_closest_element(element, elements):
    min_distance = float('inf')
    closest_element = None

    for other in elements:
        if other['id'] == element['id']:
            continue  # No comparar con el mismo elemento

        distance = calculate_distance(element['x'], element['y'], other['x'], other['y'])
        if distance < min_distance:
            min_distance = distance
            closest_element = other

    return closest_element, min_distance

def obtener_conexiones_validas(conexiones, links):
    # Creamos un conjunto vacío para almacenar las conexiones válidas
    conexiones_validas = set()
    # Iteramos sobre cada conexión
    for nodo, info in conexiones.items():
        nodo_a = nodo
        nodo_b = info['connected_to']

        # Verificamos si la conexión inversa existe
        if conexiones[nodo_b]['connected_to'] == nodo_a:
            # Verificamos si ambos nodos están conectados al mismo link
            link_a = links[nodo_a]['connected_to']
            link_b = links[nodo_b]['connected_to']

            if link_a == link_b:
                # Comprobamos si la conexión ya existe en el conjunto
                if (nodo_a, nodo_b) not in conexiones_validas and (nodo_b, nodo_a) not in conexiones_validas:
                    # Agregamos la conexión al conjunto
                    conexiones_validas.add((nodo_a, nodo_b))

    return conexiones_validas

def main(file_path):
    elements = []
    links = []

    # Leer el archivo
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1: 
                links.append({'id': f"Link_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': f"{element_type}_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    # Calcular conexiones a elementos
    connections_to_elements = {}
    for element in elements:
        closest, distance = find_closest_element(element, elements)
        if closest:
            connections_to_elements[element['id']] = {
                'connected_to': closest['id'],
                'distance': distance,
            }

    # Calcular conexiones a links
    connections_to_links = {}
    for element in elements:
        closest_link = None
        min_distance = float('inf')
        
        for link in links:
            distance = calculate_distance(element['x'], element['y'], link['x'], link['y'])
            if distance < min_distance:
                min_distance = distance
                closest_link = link['id']
                
        connections_to_links[element['id']] = {
            'connected_to': closest_link,
            'distance': min_distance
        }

    # Imprimir las conexiones
    print('Conexiones entre elementos: ')
    for key, value in connections_to_elements.items():
        print(f"{key} -> {value['connected_to']}")

    print('\nConexiones entre elementos y links:')
    for key, value in connections_to_links.items():
        print(f"{key} -> {value['connected_to']}")
            
    # Obtenemos las conexiones válidas
    conexiones_validas = obtener_conexiones_validas(connections_to_elements, connections_to_links)
    
    print("\nConexiones válidas:")
    for conexion in conexiones_validas:
        print(f"{conexion[0]} <-> {conexion[1]}")


file_path = './6.txt'
main(file_path)


Conexiones entre elementos: 
Router_1 -> Switch_6
Router_2 -> DNS_4
DNS_3 -> Router_1
DNS_4 -> Router_2
DNS_5 -> Switch_6
Switch_6 -> Router_2

Conexiones entre elementos y links:
Router_1 -> Link_11
Router_2 -> Link_9
DNS_3 -> Link_7
DNS_4 -> Link_10
DNS_5 -> Link_8
Switch_6 -> Link_9

Conexiones válidas:


- Varios links de conexión

In [11]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
   
def get_valid_connections(nearest_elements, nearest_links):
    conexiones_validas = set()

    for current_element, connected_elements in nearest_elements.items():
        current_links = nearest_links[current_element] # Links conectados al elemento actual
        
        for connected_element in connected_elements:
            connected_links = nearest_links[connected_element] # Links conectados al elemento conectado
            common_links = set(current_links) & set(connected_links) # Encontrar links comunes

            if common_links:
                connection = tuple(sorted([current_element, connected_element])) # Evitar conexiones duplicadas
                conexiones_validas.add(connection)

    return list(conexiones_validas)
 
# Encuentra el elemento más cercano
def find_valid_element(element, elements, threshold):
    valid_elements = []
    for other in elements:
        if other['id'] == element['id']:
            continue
        distance = calculate_distance(element['x'], element['y'], other['x'], other['y'])
        if distance <= threshold:
            valid_elements.append(other['id'])
    return valid_elements


def main(file_path):
    "Calculamos la distancia entre el primer elemento del txt con el resto de elementos"
    elements = []
    links = []
    connections = []

    # Leer el archivo
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1:
                links.append({'id': f"Link_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': f"{element_type}_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    # Calcular elementos cercanos
    nearest_elements = {}
    for element in elements:
        valid_elements = find_valid_element(element, elements, 0.35)
        nearest_elements[element['id']] = valid_elements       
             
    # Calcular links cercanos
    nearest_links = {}
    for element in elements:
        valid_links = find_valid_element(element, links, 0.15)
        nearest_links[element['id']] = valid_links

    # Obtenemos las conexiones válidas        
    connections= get_valid_connections(nearest_elements, nearest_links)
    
    #  Pintamos
    print('\nConexiones entre elementos:')
    for element, others in nearest_elements.items():
        for other in others:
            print(f"{element} -> {other}")
            
    print('\nConexiones entre elementos y links:')
    for element, links in nearest_links.items():
        for link in links:
            print(f"{element} -> {link}")
    
    print('\nConexiones válidas:')
    for connection in connections:
        print(f"{connection[0]} <-> {connection[1]}")

file_path = './6.txt'
main(file_path)


Conexiones entre elementos:
Router_1 -> DNS_3
Router_1 -> DNS_5
Router_1 -> Switch_6
Router_2 -> DNS_4
Router_2 -> DNS_5
Router_2 -> Switch_6
DNS_3 -> Router_1
DNS_4 -> Router_2
DNS_5 -> Router_1
DNS_5 -> Router_2
DNS_5 -> Switch_6
Switch_6 -> Router_1
Switch_6 -> Router_2
Switch_6 -> DNS_5

Conexiones entre elementos y links:
Router_1 -> Link_7
Router_1 -> Link_11
Router_2 -> Link_9
Router_2 -> Link_10
DNS_3 -> Link_7
DNS_4 -> Link_10
DNS_5 -> Link_8
Switch_6 -> Link_8
Switch_6 -> Link_9
Switch_6 -> Link_11

Conexiones válidas:
Router_2 <-> Switch_6
DNS_4 <-> Router_2
Router_1 <-> Switch_6
DNS_5 <-> Switch_6
DNS_3 <-> Router_1


- Generar JSON

In [2]:
import json
import uuid

# Función para generar una estructura de nodo basada en los datos del elemento y su tipo
def generate_node(element_id, node_type, x, y, width, height, adapters):
    node_id = str(uuid.uuid4())
    return {
        "compute_id": "local",
        "console": 5000,
        "console_auto_start": True,
        "console_type": "telnet",
        "custom_adapters": [],
        "first_port_name": None,
        "height": height,
        "label": {
            "rotation": 0,
            "style": "font-family: TypeWriter;font-size: 10.0;font-weight: bold;fill: #000000;fill-opacity: 1.0;",
            "text": element_id,
            "x": 0,
            "y": -25
        },
        "locked": True,
        "name": element_id,
        "node_id": node_id,
        "node_type": node_type,
        "port_name_format": "Ethernet{0}",
        "port_segment_size": 0,
        "properties": {
            "adapters": adapters,
            "aux": 5001,
            "console_http_path": "/",
            "console_http_port": 80,
            "console_resolution": "800x600",
            "container_id": str(uuid.uuid4()),
            "environment": f"MAC=0\nTYPE={adapters}",
            "extra_hosts": None,
            "extra_volumes": [],
            "image": "jcfabero/server:latest",
            "start_command": "/bin/bash",
            "usage": "Configure network settings."
        },
        "symbol": ":/symbols/classic/server.svg",
        "template_id": str(uuid.uuid4()),
        "width": width,
        "x": x,
        "y": y,
        "z": 1
    }

# Definir datos generales del proyecto
project_data = {
    "auto_close": True,
    "auto_open": False,
    "auto_start": False,
    "drawing_grid_size": 25,
    "grid_size": 75,
    "name": "1",
    "project_id": str(uuid.uuid4()),
    "revision": 9,
    "scene_height": 1000,
    "scene_width": 2000,
    "show_grid": False,
    "show_interface_labels": False,
    "show_layers": False,
    "snap_to_grid": False,
    "supplier": None,
    "topology": {
        "computes": [],
        "drawings": [],
        "links": [],
        "nodes": []
    },
    "type": "topology",
    "variables": None,
    "version": "2.2.29",
    "zoom": 100
}


In [3]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
   
def get_valid_connections(nearest_elements, nearest_links):
    conexiones_validas = set()

    for current_element, connected_elements in nearest_elements.items():
        current_links = nearest_links[current_element]
        
        for connected_element in connected_elements:
            connected_links = nearest_links[connected_element]
            common_links = set(current_links) & set(connected_links)

            if common_links:
                # connection = tuple(sorted([current_element, connected_element])) # Evitar conexiones duplicadas
                connection = tuple([current_element, connected_element])
                conexiones_validas.add(connection)

    return list(conexiones_validas)
 
# Encuentra el elemento más cercano en un solo bucle
def find_valid_element(element, elements, threshold):
    valid_elements = []
    for other in elements:
        if other['id'] == element['id']:
            continue
        distance = calculate_distance(element['x'], element['y'], other['x'], other['y'])
        if distance <= threshold:
            valid_elements.append(other['id'])
    return valid_elements

# Funciones JSON
def get_node_type(element_id):
    element_types = {'DNS': 'server', 'Router': 'router', 'Switch': 'switch'}
    node_type = element_types[element_id.split('_')[0]]
    return node_type


def generate_json(elements, connections):
    # Añadir nodos basados en los datos de conexión
    generated_nodes = []
    for element in connections:
        if element[0] not in generated_nodes:
            generated_nodes.append(element[0])
            element_dictionary = next(d for d in elements if d['id'] == element[0]) # Buscar dicionario del elemento en la lista de elementos
            node_type = get_node_type(element_dictionary['id'])
            x = int(element_dictionary['x'] * project_data['scene_width']) - 500
            y = int(element_dictionary['y'] * project_data['scene_height']) - 500
            node_data = generate_node(element_dictionary['id'], node_type, x, y, 70, 45, 1)
            project_data['topology']['nodes'].append(node_data)

    # Generar archivo JSON para el archivo .gns3
    json_data = json.dumps(project_data, indent=4)

    # Guardar el archivo JSON
    output_file_path = './generated_topology.gns3'
    with open(output_file_path, 'w') as file:
        file.write(json_data)
    print(output_file_path)

def main(file_path):
    "Calculamos la distancia entre el primer elemento del txt con el resto de elementos"
    elements = []
    links = []
    connections = []
    
    # Leer el archivo
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1:
                links.append({'id': f"Link_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': f"{element_type}_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    # Calcular elementos cercanos
    nearest_elements = {}
    for element in elements:
        valid_elements = find_valid_element(element, elements, 0.35)
        nearest_elements[element['id']] = valid_elements       
             
    # Calcular links cercanos
    nearest_links = {}
    for element in elements:
        valid_links = find_valid_element(element, links, 0.15)
        nearest_links[element['id']] = valid_links

    # Obtenemos las conexiones válidas        
    connections= get_valid_connections(nearest_elements, nearest_links)

    generate_json(elements, connections)
    

file_path = './2.txt'
main(file_path)

./generated_topology.gns3


# AUX

In [1]:
import json
import uuid

# Función para generar una estructura de nodo basada en los datos del elemento y su tipo
def generate_node(element_id, node_type, x, y, width, height, adapters):
    node_id = str(uuid.uuid4())
    return {
        "compute_id": "local",
        "console": 5000,
        "console_auto_start": True,
        "console_type": "telnet",
        "custom_adapters": [],
        "first_port_name": None,
        "height": height,
        "label": {
            "rotation": 0,
            "style": "font-family: TypeWriter;font-size: 10.0;font-weight: bold;fill: #000000;fill-opacity: 1.0;",
            "text": element_id,
            "x": 0,
            "y": -25
        },
        "locked": True,
        "name": element_id,
        "node_id": node_id,
        "node_type": node_type,
        "port_name_format": "Ethernet{0}",
        "port_segment_size": 0,
        "properties": {
            "adapters": adapters,
            "aux": 5001,
            "console_http_path": "/",
            "console_http_port": 80,
            "console_resolution": "800x600",
            "container_id": str(uuid.uuid4()),
            "environment": f"MAC=0\nTYPE={adapters}",
            "extra_hosts": None,
            "extra_volumes": [],
            "image": "jcfabero/server:latest",
            "start_command": "/bin/bash",
            "usage": "Configure network settings."
        },
        "symbol": ":/symbols/classic/server.svg",
        "template_id": str(uuid.uuid4()),
        "width": width,
        "x": x,
        "y": y,
        "z": 1
    }

# Definir datos generales del proyecto
project_data = {
    "auto_close": True,
    "auto_open": False,
    "auto_start": False,
    "drawing_grid_size": 25,
    "grid_size": 75,
    "name": "1",
    "project_id": str(uuid.uuid4()),
    "revision": 9,
    "scene_height": 1000,
    "scene_width": 2000,
    "show_grid": False,
    "show_interface_labels": False,
    "show_layers": False,
    "snap_to_grid": False,
    "supplier": None,
    "topology": {
        "computes": [],
        "drawings": [],
        "links": [],
        "nodes": []
    },
    "type": "topology",
    "variables": None,
    "version": "2.2.29",
    "zoom": 100
}


# Añadir nodos basados en los datos de conexión
element_types = {'DNS': 'server', 'Router': 'router', 'Switch': 'switch'}
for element in elements:
    element_id = f"{element['id']}_{elements.index(element)+1}"
    node_type = element_types[element['id']]
    x = int(element['x'] * project_data['scene_width']) - 500
    y = int(element['y'] * project_data['scene_height']) - 500
    node_data = generate_node(element_id, node_type, x, y, 70, 45, 1)
    project_data['topology']['nodes'].append(node_data)

# Generar archivo JSON para el archivo .gns3
json_data = json.dumps(project_data, indent=4)

# Guardar el archivo JSON
output_file_path = './generated_topology.gns3'
with open(output_file_path, 'w') as file:
    file.write(json_data)

output_file_path


NameError: name 'elements' is not defined